# Set-up of the ænet-LAMMPS Interface

**Example usage of the ænet-LAMMPS interface for molecular dynamics simulations with artificial neural network (ANN) potentials.**

The materials for this tutorial are available in the following GitHub repository: https://github.com/atomisticnet/psi-k-mlip-workshop-2021

**If you make use of ænet, please cite the following references as appropriate:**

A more in-depth tutorial for the construction of ANN potentials can be found in:

A.M. Miksch, T. Morawietz, J. Kästner, A. Urban, N. Artrith, “Strategies for the Construction of Machine-Learning Potentials for Accurate and Efficient Atomic-Scale Simulations”, [*Mach. Learn.: Sci. Technol.* **2** (2021) 031001](https://doi.org/10.1088/2632-2153/abfd96). 

The atomic energy network (ænet) package: http://ann.atomistic.net

[N. Artrith and A. Urban, *Comput. Mater. Sci.* **114** (2016) 135-150.](http://dx.doi.org/10.1016/j.commatsci.2015.11.047)<br/>
[N. Artrith, A. Urban, and G. Ceder, *Phys. Rev. B* **96** (2017) 014112.](http://dx.doi.org/10.1103/PhysRevB.96.014112)

The ænet-LAMMPS interface:

[M. S. Chen, T. Morawietz, H. Mori, T. E. Markland, and N. Artrith, *J. Chem. Phys.* **155** (2021) 074801.](https://doi.org/10.1063/5.0063880)

The atomic simulation environment (ASE): https://wiki.fysik.dtu.dk/ase/

[A. H. Larsen et al., *J. Phys.: Condens. Matter* **29** (2017) 273002.](https://doi.org/10.1088/1361-648X/aa680e)

The Large-scale Atomic/Molecular Massively Parallel Simulator (LAMMPS): https://www.lammps.org

[A. P. Thompson et al., Comp. Phys. Commun. 271 (2022) 10817.](https://doi.org/10.1016/j.cpc.2021.108171)

Contact: N. Artrith (n.artrith@uu.nl), M. S. Chen (misch@stanford.edu), A. M. Miksch (miksch@theochem.uni-stuttgart.de), A. Urban (a.urban@columbia.edu)

## Downloading and installing LAMMPS

For this example, we will be using the <a href="https://www.lammps.org/">LAMMPS</a> release from 4 Feb 2020. Running the cell below downloads the software package into the working directory.

In [ ]:
!wget https://download.lammps.org/tars/lammps-4Feb2020.tar.gz
!tar -xzvf lammps-4Feb2020.tar.gz

lammps-4Feb20/src/USER-MOLFILE/Install.sh
lammps-4Feb20/src/USER-MOLFILE/molfile_interface.cpp
lammps-4Feb20/src/USER-MOLFILE/dump_molfile.cpp
lammps-4Feb20/src/USER-MOLFILE/reader_molfile.cpp
lammps-4Feb20/src/USER-MOLFILE/reader_molfile.h
lammps-4Feb20/src/fix_wall_region.h
lammps-4Feb20/src/compute_pe_atom.cpp
lammps-4Feb20/src/npair_full_nsq_ghost.h
lammps-4Feb20/src/Makefile
lammps-4Feb20/src/dump_atom.cpp
lammps-4Feb20/src/fix_wall_harmonic.cpp
lammps-4Feb20/src/nstencil_half_bin_3d_newton_tri.h
lammps-4Feb20/src/pair_lj_smooth_linear.h
lammps-4Feb20/src/npair_full_bin_ghost.h
lammps-4Feb20/src/npair_half_respa_bin_newtoff.h
lammps-4Feb20/src/thermo.h
lammps-4Feb20/src/USER-AWPMD/
lammps-4Feb20/src/USER-AWPMD/atom_vec_wavepacket.cpp
lammps-4Feb20/src/USER-AWPMD/README
lammps-4Feb20/src/USER-AWPMD/pair_awpmd_cut.h
lammps-4Feb20/src/USER-AWPMD/fix_nve_awpmd.cpp
lammps-4Feb20/src/USER-AWPMD/Install.sh
lammps-4Feb20/src/USER-AWPMD/atom_vec_wavepacket.h
lammps-4Feb20/src/USER-AWPMD/fi

Now we download the ænet-LAMMPS code directly from the GitHub repository.

In [ ]:
!git clone https://github.com/atomisticnet/aenet-lammps.git

fatal: destination path 'aenet-lammps' already exists and is not an empty directory.


Next, we install LAMMPS. For more detailed instructions on how to patch and compile LAMMPS with ænet support please see the <a href="https://github.com/atomisticnet/aenet-lammps">ænet-LAMMPS GitHub repository</a>.

First, let us organize things so that the ænet header and library files are where we expect them to be when compiling things.

In [ ]:
%cd /work/lammps-4Feb20/lib/
!mkdir -p aenet/lib
!mkdir -p aenet/include
%cd /work/lammps-4Feb20/lib/aenet/include
!ln -s /work/aenet/src/aenet.h
%cd /work/lammps-4Feb20/lib/aenet/lib
!ln -s /work/aenet/src/libaenet.a
!ln -s /work/aenet/lib/liblbfgsb.a
!ln -s /work/aenet/src/libaenet.so
!ln -s /work/aenet/lib/liblbfgsb.so

/work/lammps-4Feb20/lib
/work/lammps-4Feb20/lib/aenet/include
ln: failed to create symbolic link './aenet.h': File exists
/work/lammps-4Feb20/lib/aenet/lib
ln: failed to create symbolic link './libaenet.a': File exists
ln: failed to create symbolic link './liblbfgsb.a': File exists
ln: failed to create symbolic link './libaenet.so': File exists
ln: failed to create symbolic link './liblbfgsb.so': File exists


Next, let us patch LAMMPS with the ænet-LAMMPS interface files. This entails copying over the provided `USER-AENET` folder into the LAMMPS `src` directory and replacing the LAMMPS `Makefile`. Since we compiled the ænet library using `Makefile.gfortran_openblas_serial`, we will use the corresponding `Install.sh` file for our custom `USER-AENET` package so as to make sure that OpenBLAS is linked during the compilation.

In [ ]:
%cd /work/lammps-4Feb20/src/
! cp -r /work/aenet-lammps/USER-AENET .
! cp -f /work/aenet-lammps/Makefile-aenetlammps Makefile
! cp -f USER-AENET/Install.sh-gfortran_serial USER-AENET/Install.sh
#! cp -f USER-AENET/Install.sh-gfortran_openblas_serial USER-AENET/Install.sh

/work/lammps-4Feb20/src


We need to flag the `USER-AENET` package to be included as part of our LAMMPS compilation, and lastly we compile an MPI-parallelized LAMMPS executable.

**Note: This can take a few minutes.**

In [ ]:
%cd /work/lammps-4Feb20/src/
! make yes-user-aenet
! make mpi

/work/lammps-4Feb20/src
Installing package user-aenet
Gathering installed package information (may take a little while)
make[1]: Entering directory '/work/lammps-4Feb20/src'
Gathering git version information
make[1]: Leaving directory '/work/lammps-4Feb20/src'
Compiling LAMMPS for machine mpi
make[1]: Entering directory '/work/lammps-4Feb20/src/Obj_mpi'
make[1]: Leaving directory '/work/lammps-4Feb20/src/Obj_mpi'
make[1]: Entering directory '/work/lammps-4Feb20/src/Obj_mpi'
mpicxx -g -O3   -DLAMMPS_GZIP -DLAMMPS_MEMALIGN=64   -I../../lib/aenet/include  -DMPICH_SKIP_MPICXX -DOMPI_SKIP_MPICXX=1    -c ../lammps.cpp
mpicxx -g -O3 -L../../lib/aenet/lib      nstencil_half_multi_2d_newton.o fix_wall.o compute_chunk_atom.o min.o fix_print.o math_extra.o nstencil_half_bin_3d_newtoff.o ntopo_bond_template.o pair_yukawa.o neigh_request.o comm.o atom_vec_charge.o compute_deprecated.o ntopo_angle_partial.o compute_rdf.o pair_lj_cut_coul_dsf.o compute_heat_flux.o npair_full_multi.o compute_pressure.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=bfabd374-f7a6-40fd-b4f0-3c25f442361e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>